In [1]:
!nvidia-smi -L

GPU 0: NVIDIA A100-SXM4-80GB (UUID: GPU-6620e1a6-0d7f-6cfe-1290-c3b795ee422b)
  MIG 1g.10gb     Device  0: (UUID: MIG-23c4e088-ce35-5324-afdf-34f3960bcdb0)
  MIG 1g.10gb     Device  1: (UUID: MIG-a40ffb87-88e6-5f6f-bde7-1af6ad01adf2)
  MIG 1g.10gb     Device  2: (UUID: MIG-fccb8438-5870-52c3-aafa-42fddedd6b8e)
  MIG 1g.10gb     Device  3: (UUID: MIG-8e6a7989-84f7-53a8-ac83-762afe1fbd36)
  MIG 1g.10gb     Device  4: (UUID: MIG-b71e4b1a-a7c6-55b2-bdee-ddfc11df3f60)
  MIG 1g.10gb     Device  5: (UUID: MIG-1f8b2754-cb9d-509f-b2dc-c4ff99a35290)
  MIG 1g.10gb     Device  6: (UUID: MIG-f7abf2c0-a5f2-5ca3-b7cb-215c92d6e9c9)
GPU 1: NVIDIA A100-SXM4-80GB (UUID: GPU-e84e0c4d-caeb-bff4-5bc2-79000e81f2c0)
  MIG 1g.10gb     Device  0: (UUID: MIG-65568ef3-874c-554e-8033-47d7eda53db5)
  MIG 1g.10gb     Device  1: (UUID: MIG-f31fdd03-d04b-53cb-b889-3c53ba7e4c29)
  MIG 1g.10gb     Device  2: (UUID: MIG-e76da236-a0d8-513c-9b81-253e7b9bda25)
  MIG 1g.10gb     Device  3: (UUID: MIG-248dba60-a9de-5218-81ae-

In [2]:
!nvidia-smi

Sat Apr  6 00:01:01 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.07             Driver Version: 535.161.07   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  | 00000000:07:00.0 Off |                   On |
| N/A   29C    P0              56W / 400W |     87MiB / 81920MiB |     N/A      Default |
|                                         |                      |              Enabled |
+-----------------------------------------+----------------------+--

In [2]:
import os
import torch

# Set CUDA_VISIBLE_DEVICES to a specific 7g.80gb MIG device
#os.environ["CUDA_VISIBLE_DEVICES"] = "1:0"  # Use MIG 7g.80gb Device 0 under GPU 6

# Check the available GPU devices
if torch.cuda.is_available():
    num_devices = torch.cuda.device_count()
    print(f"Number of available GPU devices: {num_devices}")
    for i in range(num_devices):
        device_name = torch.cuda.get_device_name(i)
        device_properties = torch.cuda.get_device_properties(i)
        total_memory = device_properties.total_memory / (1024 * 1024)  # Convert bytes to MB
        device = torch.device(f"cuda:{i}")
        print(f"Device {i}: {device}")
        print(f"Device {i}: {device_name}")
        print(f"  Total GPU Memory: {total_memory} MB")
else:
    print("CUDA is not available")

Number of available GPU devices: 1
Device 0: cuda:0
Device 0: NVIDIA A100-SXM4-80GB MIG 7g.80gb
  Total GPU Memory: 81050.625 MB


In [2]:
import os

# Set CUDA_VISIBLE_DEVICES
os.environ["CUDA_VISIBLE_DEVICES"] = "0:0"  # Set visible devices to GPU 0 and 1

# Rest of your code
import torch

if torch.cuda.is_available():
    num_devices = torch.cuda.device_count()
    print(f"Number of visible GPU devices: {num_devices}")
    # Additional code to utilize the GPU(s)
else:
    print("CUDA is not available")





# Check if CUDA is available
if torch.cuda.is_available():
    # Get the number of available GPUs
    num_gpus = torch.cuda.device_count()
    print(f"Number of available GPUs: {num_gpus}")

    # Iterate over each GPU and display its specifications
    for gpu_id in range(num_gpus):
        print(f"\nGPU {gpu_id} specifications:")
        
        # Set the current device
        torch.cuda.set_device(gpu_id)
        
        # Get the device properties
        device_properties = torch.cuda.get_device_properties(gpu_id)
        
        # Display the device name
        print(f"Device name: {device_properties.name}")
        
        # Display the total memory
        total_memory = device_properties.total_memory / (1024 * 1024)  # Convert bytes to MB
        print(f"Total memory: {total_memory:.2f} MB")
        
        # Display the major and minor compute capability
        major, minor = device_properties.major, device_properties.minor
        print(f"Compute capability: {major}.{minor}")
        
        # Display the number of multiprocessors
        num_multiprocessors = device_properties.multi_processor_count
        print(f"Number of multiprocessors: {num_multiprocessors}")
        
      
else:
    print("CUDA is not available. Please check your PyTorch installation and GPU setup.")

Number of visible GPU devices: 1
Number of available GPUs: 1

GPU 0 specifications:
Device name: NVIDIA A100-SXM4-80GB MIG 7g.80gb
Total memory: 81050.62 MB
Compute capability: 8.0
Number of multiprocessors: 98


In [2]:
import os

cuda_visible_devices = os.environ.get("CUDA_VISIBLE_DEVICES")
if cuda_visible_devices:
    print(f"CUDA_VISIBLE_DEVICES: {cuda_visible_devices}")
else:
    print("CUDA_VISIBLE_DEVICES is not set")

CUDA_VISIBLE_DEVICES: 0:0


In [4]:
import os

num_cores = os.cpu_count()
print(f"Total number of CPU cores: {num_cores}")

Total number of CPU cores: 256


In [3]:
import torch
import torch.nn as nn

# Specify the GPU to use (e.g., 0 for the first GPU)
gpu_id = 0

# Set the device to use
device = torch.device(f"cuda:{gpu_id}" if torch.cuda.is_available() else "cpu")

# Define a simple neural network
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(100, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return x

# Create an instance of the model
model = SimpleNet()

# Move the model to the specified device
model.to(device)

# Generate dummy input data
batch_size = 200
input_data = torch.randn(batch_size, 100).to(device)

# Perform a forward pass through the model
output = model(input_data)

print("Input shape:", input_data.shape)
print("Output shape:", output.shape)
print("Model running on:", device)

Input shape: torch.Size([200, 100])
Output shape: torch.Size([200, 10])
Model running on: cuda:0


In [7]:
import torch
import torch.nn as nn

# Specify the GPU to use (e.g., 0 for the first GPU)
gpu_id = 0

# Set the device to use
device = torch.device(f"cuda:{gpu_id}" if torch.cuda.is_available() else "cpu")

# Define a simple neural network
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(100, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return x

# Create an instance of the model
model = SimpleNet()

# Move the model to the specified device
model.to(device)

# Generate dummy input data
batch_size = 200
input_data = torch.randn(batch_size, 100).to(device)

# Perform a forward pass through the model
output = model(input_data)

print("Input shape:", input_data.shape)
print("Output shape:", output.shape)
print("Model running on:", device)

Input shape: torch.Size([200, 100])
Output shape: torch.Size([200, 10])
Model running on: cuda:0


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Check if GPU is available and set PyTorch device
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

# Sample neural network model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(10, 50)  # 10 input features, 50 outputs
        self.fc2 = nn.Linear(50, 1)   # 50 input features, 1 output feature

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net().to(device)  # Transfer model to GPU

# Create a simple dataset and dataloader
x = torch.randn(100, 10)  # 100 samples, 10 features
y = torch.randn(100, 1)   # 100 samples, 1 target variable
dataset = TensorDataset(x, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
for epoch in range(10):  # Loop over the dataset multiple times
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)  # Transfer data to GPU

        optimizer.zero_grad()   # Zero the parameter gradients

        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(dataloader)}")

print('Finished Training')


In [ ]:
import subprocess
import os

# GPU UUID you want to use
target_uuid = "MIG-b1700e4f-1b5c-53c6-b75e-b52536ff7060"

# Get the list of GPUs and their UUIDs
output = subprocess.check_output("nvidia-smi -L", shell=True).decode()

# Find the logical ID for the desired GPU UUID
for line in output.split('\n'):
    if target_uuid in line:
        gpu_id = line.split(':')[0].split(' ')[1]
        break

# Set the environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_id

# Now import PyTorch and continue with your script
import torch


In [ ]:
gpu_id

In [ ]:
output

In [ ]:
import subprocess

def get_gpu_memory_by_uuid(target_uuid):
    # Execute the nvidia-smi command to get UUID and memory info
    cmd = "nvidia-smi --query-gpu=uuid,memory.total --format=csv"
    output = subprocess.check_output(cmd, shell=True).decode()

    # Parse the output to find the memory for the target UUID
    for line in output.split('\n'):
        if target_uuid in line:
            # The memory info is expected to be in the second column
            memory_info = line.split(',')[1].strip()
            return memory_info
    return "GPU not found"

# Replace this with your GPU's UUID
target_uuid = "GPU-901a28b5-9684-0cc5-cec6-d78b14cfe130"
memory_info = get_gpu_memory_by_uuid(target_uuid)
print(memory_info)


In [8]:
import torch

print(torch.__path__)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA version: {torch.version.cuda}")
print(f"CUDNN version: {torch.backends.cudnn.version()}")
print(f"Number of available GPUs: {torch.cuda.device_count()}")
# print(f"Current GPU: {torch.cuda.current_device(1)}")
# print(f"CUDA device name: {torch.cuda.get_device_name(0)}")
# print(f"CUDA capability: {torch.cuda.get_device_capability()}")
print(f"CUDA memory allocated: {torch.cuda.memory_allocated() / (1024**3):.2f} GB")
print(f"CUDA memory cached: {torch.cuda.memory_reserved() / (1024**3):.2f} GB")



['/home/tuchsanai/.conda/envs/torch/lib/python3.12/site-packages/torch']
PyTorch version: 2.2.2+cu121
CUDA version: 12.1
CUDNN version: 8902
Number of available GPUs: 1
CUDA memory allocated: 0.01 GB
CUDA memory cached: 0.02 GB


In [ ]:
!pip install transformers datasets evaluate

# clean up the output
from IPython.display import clear_output
clear_output()

In [ ]:
device = torch.device("cuda:7" if torch.cuda.is_available() else "cpu")
device

In [ ]:
from datasets import load_dataset
from transformers import AutoFeatureExtractor, AutoModelForImageClassification, TrainingArguments, Trainer
from PIL import Image
import torch
from torchvision import transforms
import matplotlib.pyplot as plt

# Load the dataset
dataset = load_dataset("beans")

dataset

In [ ]:
labels = dataset["train"].features["labels"].names


label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

In [ ]:
labels 

In [ ]:
label2id,

In [ ]:
id2label

In [ ]:


# Load the pre-trained model and feature extractor
model_name = "google/vit-base-patch16-224-in21k"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
model = AutoModelForImageClassification.from_pretrained(model_name,  num_labels=len(labels))

model


In [ ]:
model 

In [ ]:
dataset['train'][0]['image']

In [ ]:
dataset['train'][0]['labels']

In [ ]:

# Define the image augmentation transforms
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std),
])

val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std),
])

# Preprocess the dataset with image augmentation
def preprocess_function(examples):
    inputs = {}
    inputs["pixel_values"] = [train_transforms(image.convert("RGB")) for image in examples["image"]]
    inputs["labels"] = examples["labels"]
    return inputs

def preprocess_function_val(examples):
    inputs = {}
    inputs["pixel_values"] = [val_transforms(image.convert("RGB")) for image in examples["image"]]
    inputs["labels"] = examples["labels"]
    return inputs

processed_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)
processed_dataset["validation"] = dataset["validation"].map(preprocess_function_val, batched=True, remove_columns=dataset["validation"].column_names)

# Display

In [ ]:
processed_dataset 

In [ ]:
# Display some augmented images
fig, axs = plt.subplots(2, 4, figsize=(12, 6))
for i in range(2):
    for j in range(4):
        image = processed_dataset["train"][i*4+j]["pixel_values"]
        image = torch.tensor(image)  # Convert the list to a tensor
        image = image.permute(1, 2, 0).numpy()
        image = (image - image.min()) / (image.max() - image.min())  # Normalize pixel values to [0, 1]
        axs[i, j].imshow(image)
        axs[i, j].axis("off")
plt.tight_layout()
plt.show()

In [ ]:
import psutil
import torch
from transformers import Trainer
from transformers import TrainerCallback


def get_cpu_gpu_usage():
    cpu_usage = psutil.cpu_percent()
    gpu_usage = torch.cuda.utilization() if torch.cuda.is_available() else None
    return cpu_usage, gpu_usage


class MonitoringCallback(TrainerCallback):
    def __init__(self):
        self.cpu_gpu_usage = []

    def on_step_end(self, args, state, control, **kwargs):
        cpu_usage, gpu_usage = get_cpu_gpu_usage()
        self.cpu_gpu_usage.append((cpu_usage, gpu_usage))

In [ ]:

# Define the data collator
def collate_fn(examples):
    pixel_values = torch.stack([torch.tensor(example["pixel_values"]) for example in examples])
    labels = torch.tensor([example["labels"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=96,
    evaluation_strategy="steps",
    num_train_epochs=5,
    #fp16=True,
    
    save_steps=100,
    eval_steps=100,
    logging_steps=10,
    learning_rate=2e-5,
    save_total_limit=2,
    remove_unused_columns=False,
    metric_for_best_model="accuracy",
)

# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["validation"],
    tokenizer=feature_extractor,
    data_collator=collate_fn,
   
)

# Train the model
trainer.train()

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Check if GPU is available and set PyTorch device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Sample neural network model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(10, 50)  # 10 input features, 50 outputs
        self.fc2 = nn.Linear(50, 1)   # 50 input features, 1 output feature

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net().to(device)  # Transfer model to GPU

# Create a simple dataset and dataloader
x = torch.randn(100, 10)  # 100 samples, 10 features
y = torch.randn(100, 1)   # 100 samples, 1 target variable
dataset = TensorDataset(x, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
for epoch in range(10):  # Loop over the dataset multiple times
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)  # Transfer data to GPU

        optimizer.zero_grad()   # Zero the parameter gradients

        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(dataloader)}")

print('Finished Training')


Epoch 1, Loss: 0.8551386669278145
Epoch 2, Loss: 0.8150906972587109
Epoch 3, Loss: 1.0423409044742584
Epoch 4, Loss: 1.350922852754593
Epoch 5, Loss: 0.8309062048792839
Epoch 6, Loss: 0.8482799157500267
Epoch 7, Loss: 1.0287215113639832
Epoch 8, Loss: 0.9033944606781006
Epoch 9, Loss: 0.8291004002094269
Epoch 10, Loss: 0.8488811552524567
Finished Training
